# P4 Assignment

For this assignment, you will be writing P4 code which enables a switch to function as a joint router and firewall for an IPv6 network. The network topology is shown below.
<img src="./images/assignment4_topology.png" width="1000px">

Note: h23 will only be configured for a final test case, and begins unconfigured.

Authored by Alexander Wolosewicz, IIT, adapting some code by Nishanth Shyamkumar, IIT


## Create topology
The creation of the topology is done automatically for you. If you ever close the Jupyter kernel, or it crashes, all variable values are lost. The below cell can be executed to reset all the variable values.
Note that if the slice expires (24 hours after creation) you must start from the beginning. Any progress made in files (the shell script or P4 code) is safe as long as it is saved.
Make sure to retype your student ID for "student_id".

In [2]:
student_id = "btursunbayev"
verbose = False

In [4]:
# Setup slice, or retrieve values if slice already exists
from scripts import funcs

slice_values = funcs.setup(student_id=student_id, verbose=verbose)

vslice = slice_values['slice']
configs = slice_values['configs']
ips = slice_values['ips']
macs = slice_values['macs']
ifaces = slice_values['ifaces']
subnets = slice_values['subnets']
hosts = slice_values['hosts']
switches = slice_values['switches']
gateways = slice_values['gateways']


Retry: 13, Time: 335 sec



Time to print interfaces 352 seconds
Installing needed tools
1/8 installs finished
2/8 installs finished
3/8 installs finished
4/8 installs finished
5/8 installs finished
6/8 installs finished
7/8 installs finished
8/8 installs finished
1/8 logging jobs finished
2/8 logging jobs finished
3/8 logging jobs finished
4/8 logging jobs finished
5/8 logging jobs finished
6/8 logging jobs finished
7/8 logging jobs finished
8/8 logging jobs finished
1/8 upload jobs finished
2/8 upload jobs finished
3/8 upload jobs finished
4/8 upload jobs finished
5/8 upload jobs finished
6/8 upload jobs finished
7/8 upload jobs finished
8/8 upload jobs finished
Finished installing needed tools
Configuring node interfaces
Configuring s1...Done
Configuring s2...Done
Configuring h11...Done
Configuring h12...Done
Configuring h13...Done
Configuring h21...Done
Configuring h22...Done
Configuring h23...Done


1/14 address configuration jobs finished
2/14 address configuration jobs finished
3/14 address configuration 

In [5]:
# If the above had any node fail, UNCOMMENT the line below and execute this cell, then try again
# slice.delete()

# P4 Programming
The main objective of this assignment is to create P4 code which will configure the switches (s1 and s2 in the topology) to function as IPv6 routers and firewalls. A copy of the P4 program from the tutorial, which created an IPv4 router, is provided as a starter template in [assignment.p4](./assignment.p4).

The routing requirements are that:
- hosts on each subnet can ping each other
- hosts can ping hosts on the other subnet, and the switches route to the other subnet using LPM

The firewall requirement is that TCP or UDP packets must have one of the ports listed below as either the source or destination port.

Besides modifying P4 code, you will also need to edit [add_rules_s1.sh](./add_rules_s1.sh) and [add_rules_s2.sh](./add_rules_s2.sh) to contain the control plane commands the switches need to be configured with.

In a Jupyter cell, `funcs.start_switches(switches, verbose)` and `funcs.stop_switches(switches, verbose)` will respectively start and stop the switches (see the start and end of the Testing section).

In [6]:
# For convenience, list all the values from the config file
funcs.print_values(configs, macs, ips)

s1's MAC addresses are (in order from port 0 to port 3):
1c:88:18:16:d2:41
04:66:be:6a:81:dc
d0:71:79:3d:7e:0a
80:ce:01:83:8c:e4
s2's MAC addresses are (in order from port 0 to port 3):
c4:9b:87:6c:82:15
d4:bb:4e:80:f5:23
6c:4c:de:68:89:30
20:2f:38:b6:87:8a
For the above, s1 and s2 are connected to each other on their port 0. Port 1 connects to h11 or h21, port 2 to h12 or h22, and port 3 to h13 or h23.
h11's MAC address is f0:b6:f2:1f:4e:1b and its IPv6 address is 2001:db8:28fe:5613::3e2d:df8e
h12's MAC address is 7c:ee:f9:5d:ba:3a and its IPv6 address is 2001:db8:28fe:5613::fb6c:822
h13's MAC address is e8:a8:94:61:ad:f8 and its IPv6 address is 2001:db8:28fe:5613::c28e:73e3
h21's MAC address is 40:ba:9b:57:c3:7a and its IPv6 address is 2001:db8:347c:d902::840:5a3b
h22's MAC address is dc:56:0d:1e:81:15 and its IPv6 address is 2001:db8:347c:d902::9610:d3ec
The allowed TCP ports are: 517 648 676 849
The allowed UDP ports are: 45 178 885 988
A reminder that in the topology, h23 is ignor

# SSH Keys
The below cell will print the ssh keys, which you can enter into a new terminal tab to ssh to that client. Useful for testing/debugging your P4 code by:
<list>
    <li> SSHing into two terminals for s1 - run ./run_bm2_s1.sh in s1, let it start, then ./add_rules_s1.sh, and the same for s2 </li>
    <li> SSHing into a host and attempting to ping another with ping6, or using send.py to send a packet with specific TCP/UDP ports </li>
</list>

In [7]:
for node in vslice.get_nodes():
    print(f"{node.get_name()}: {node.get_ssh_command()}")

h11: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fee2:9435
h12: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fed1:442b
h13: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:feea:3e5d
s1: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe93:875d
h21: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe20:2977
h22: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3eff:fe3f:87b7
h23: ssh -i /home/fabric/work/fabric_config/slice_key -F /home/fabric/work/fabric_config/ssh_config ubuntu@2610:1e0:1700:206:f816:3

# Testing
The below cells will automatically test your P4 and sh scripts. The score breakdown for this assignment is:
- 2.5pts for h11 through h21 being able to ping each other
- 5 pts for the switches correctly filtering TCP and UDP ports
- 2.5pts for correctly routing to a random host added to subnet 2

There is a total of 10 points. These cells should not be modified, and they should be run in order.

In [8]:
# Start switches before the tests - this also uploads your most recent P4 and add_rules files
funcs.start_switches(switches, verbose)

Starting switches
Compiling P4...


Compiling P4...


Switches started


In [ ]:
# Ping Test
pingscore = funcs.ping_test(hosts, ips)

h11 failed to ping h12
h11 failed to ping h13
h11 failed to ping h21
h11 failed to ping h22
h11 0/4 pings successful, 0pts
h12 failed to ping h11
h12 failed to ping h13
h12 failed to ping h21
h12 failed to ping h22
h12 0/4 pings successful, 0pts
h13 failed to ping h11
h13 failed to ping h12
h13 failed to ping h21
h13 failed to ping h22
h13 0/4 pings successful, 0pts
h21 failed to ping h11
h21 failed to ping h12
h21 failed to ping h13
h21 failed to ping h22
h21 0/4 pings successful, 0pts
h22 failed to ping h11
h22 failed to ping h12
h22 failed to ping h13


In [ ]:
# Port Test
portscore = funcs.port_test(hosts, ips, configs, ifaces, student_id, verbose)

In [ ]:
# h23 Test
h23score = funcs.h23_test(hosts, ips, switches, ifaces, configs, macs, gateways, verbose)

print(f"Total Score: {pingscore+portscore+h23score}/10pts")

In [ ]:
# To stop the switches, so they can be restarted (such as to update their P4 code)
funcs.stop_switches(switches, verbose)

In [ ]:
# Run this to generate the .tar.gz file you will submit to Canvas
funcs.create_tar(student_id, hosts, switches, override=True)

In [ ]:
# Uncomment and delete your slice when you're finished
# vslice.delete()